# INSURANCE DASHBOARD

In [28]:
# import modules
import pandas as pd
import matplotlib as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact
import seaborn as sns

%matplotlib inline

In [29]:
# load dataset
df = pd.read_csv('insurance.csv')

# explore dataset
#df.info()

In [30]:
# filter data by charges
# ALL will allow the user to remove the filter
ALL = 'ALL'

def unique_sorted_values_plus_ALL(array):
    """
    Will allow the user to select uninque value
    when filtering the data or seel all the data
    """
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [31]:
# create dropdowns to filter data frame by selected value
dropdown_bmi = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.bmi), description='BMI')

# capture widget out
output_bmi = widgets.Output()
plot_output = widgets.Output()
       
def dropdown_bmi_event(change):
    """Will allow user to access info
    based on whether the selection is ALL or
    unique value"""
    plot_output.clear_output()
    output_bmi.clear_output() #clears the previous output on each iteration
    with output_bmi:
        if (change.new == ALL):
            display(df)
        else:
            display(df[df.bmi == change.new])
    with plot_output:
        @interact(hue=['sex', 'region', 'smoker'])
        def plot(hue):
            plot_output = sns.pairplot(df, hue=hue)
        

         
# link the dropdown to the observe method which will take the invoked function filter through the data frame
dropdown_bmi.observe(dropdown_bmi_event, names='value')

display(dropdown_bmi)

Dropdown(description='BMI', options=('ALL', 15.96, 16.815, 17.195, 17.29, 17.385, 17.4, 17.48, 17.67, 17.765, …

In [32]:
#display(output_bmi)
tab = widgets.Tab([output_bmi, plot_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'Pair Plot')
display(tab)